<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js018_periodicPSD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js018_periodicPSD2D / def exec_html_js()

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js018_periodicPSD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js018_periodicPSD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.12 created, last updated on 2018.11.15
//    ver 0.0.1  2019.01.13 v1, last updated on 2021.05.07
//    ver 0.0.2  2021.10.29 v2, last updated on 2021.10.29
//    ver 0.0.3  2023.03.06 v3, last updated on 2023.08.16
//
// --------------------  steepest descent method 2D
//
//  system Hamiltonian: H = -delta/2 + V(r) , delta r = div grad r
//  eigen energy set { Ei }, eigen function set { |i> }
//
//  procedure : successive approximation
//   (i) trial function set { |0>,|1>,..,|i>,.. }
//   (2) energy of |i> : ei = <i|H|i>/<i|i>
//   (3) steepest gradient direction (H-ei)|i>
//   (4) next generation : |i(next)> = |i> - dampingFactor*(H-ei)|i>
//   (5) orthogonalization { |0>,|1>,..,|i>,.. }  (Gram-Schmidt)
//   (6) sort state (It is not always necessary)
//   goto (2)
//
//  periodic condition
//
// --------------------
*/

const periodicPSD2D = (function(){ // ====================  periodicPSD2D Module  ====================

	const g_auLength = 5.29177211e-11;				// (m) 1(au) = auLength (m), (au: atomic unit hBar=1,e=1,me=1,a0=1)
	const g_auTime = 2.418884326e-17;				// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;				// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;						// (eV) 1(au) = 27.211386 (eV)
	const g_nMax = 128;								// Max(NNx,NNy)

	let g_iterCount = 0;							// sd iteration count
	let g_NNx = 64;									// xMax = NNx*dx
	let g_NNy = 64;									// yMax = NNy*dy
	let g_dx = 1.0/4.0;								// (au) x-division
	let g_dy = 1.0/4.0;								// (au) y-division
	let g_dampingFactor = 0.01;						// steepest descent damping factor

	const g_sdEnergy = dim1( 20 );					// sdEnergy[20] electron state energy
	const g_sdState = dim3( 20, g_nMax, g_nMax );	// sdState[20][128][128] electron state 0...19
	const g_wrk = dim2( g_nMax, g_nMax );			// wrk[128][128] state work space in steepestDescent
	const g_vv = dim2( g_nMax, g_nMax );			// vv[128][128] external potential

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dim3(ni,nj,nk) {
		let a=[];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( stateMax, vIndex ) {
		g_iterCount = 0;
		setInitialState(stateMax);
		setPotential(vIndex);
	}

	function setInitialState(stateMax) {
		const nnx=g_NNx, nny=g_NNy;
		for (let ist=0; ist<stateMax; ist++) {
			for (let i=0; i<nnx; i++) {
				for (let j=0; j<nny; j++) {
					g_sdState[ist][i][j] = Math.random()-0.5;
				}
			}
			normalizeState(ist);
		}
	}

	function setPotential(vIndex) {
		const nnx=g_NNx, nny=g_NNy, x0 = 0.5*nnx*g_dx, y0 = 0.5*nny*g_dy;
		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			for (let j=0; j<nny; j++) {
				const y = j*g_dy;
				const r2 = ((x-x0)*(x-x0)+(y-y0)*(y-y0));
				if (vIndex==0) { // 0:free space
					g_vv[i][j] = 0.0;
				} else if (vIndex==1) { // 1:checker board
					g_vv[i][j] = ((x-x0)*(y-y0)>0) ? 2.0 : 0.0;
				}  else if (vIndex==2) { // 1:quasi-1d periodic
					g_vv[i][j] = (Math.abs(y-y0)<=1) ? 0.0 : 20.0;
				}
			}
		}
	}


	// --------------------  steepest descent iteration  --------------------

	function SDiteration( stateMax, iterMax ) {
		for (let i=0; i<iterMax; i++) {
			for (let ist=0; ist<stateMax; ist++) {
				g_sdEnergy[ist] = steepestDescent(ist, g_dampingFactor);
			}
			GramSchmidt(stateMax);
			sortState(stateMax); // It is not always necessary
			g_iterCount += 1;
		}
	}

	function steepestDescent(ist,damp) {
		const nnx=g_NNx, nny=g_NNy, h2 = 2.0*g_dx*g_dx, fai=g_sdState[ist];
		const ei = energyOfState(ist);
		for (let i=0; i<nnx; i++) {
			const ip1 = (i+1)%nnx, im1 = (i-1+nnx)%nnx;
			for (let j=0; j<nny; j++) {
				const jp1 = (j+1)%nny, jm1 = (j-1+nny)%nny;
				g_wrk[i][j] = (4*fai[i][j]-fai[ip1][j]-fai[im1][j]-fai[i][jp1]-fai[i][jm1])/h2
								+ (g_vv[i][j]-ei)*fai[i][j];
			}
		}
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_sdState[ist][i][j] -= damp*g_wrk[i][j];
			}
		}
		normalizeState(ist);
		return ei;
	}

	function energyOfState(ist) {
		const nnx=g_NNx, nny=g_NNy, h2 = 2.0*g_dx*g_dx, fai=g_sdState[ist];
		let s = 0.0, sn = 0.0;
		for (let i=0; i<nnx; i++) {
			const ip1 = (i+1)%nnx, im1 = (i-1+nnx)%nnx;
			for (let j=0; j<nny; j++) {
				const jp1 = (j+1)%nny, jm1 = (j-1+nny)%nny;
				s += fai[i][j]*( (4*fai[i][j]-fai[ip1][j]-fai[im1][j]-fai[i][jp1]-fai[i][jm1])/h2
						+ g_vv[i][j]*fai[i][j]);
				sn += fai[i][j]*fai[i][j];
			}
		}
		return s/sn;
	}

	function GramSchmidt(stateMax) {
		const nnx=g_NNx, nny=g_NNy;
		normalizeState(0);
		for (let istate=1; istate<stateMax; istate++) {
			for (let ist=0; ist<istate; ist++) {
				const s = innerProduct(ist,istate);
				for (let i=0; i<nnx; i++) {
					for (let j=0; j<nny; j++) {
						g_sdState[istate][i][j] -= s*g_sdState[ist][i][j];
					}
				}
			}
			normalizeState(istate);
		}
	}

	function sortState(stateMax) {
		const nnx=g_NNx, nny=g_NNy;
		for (let ist=stateMax-2; ist>=0; ist--) {
			if (g_sdEnergy[ist]>g_sdEnergy[ist+1]+0.00001) {
				for (let i=0; i<nnx; i++) {
					for (let j=0; j<nny; j++) {
						const w = g_sdState[ist][i][j];
						g_sdState[ist][i][j] = g_sdState[ist+1][i][j];
						g_sdState[ist+1][i][j] = w;
					}
				}
				const w = g_sdEnergy[ist];
				g_sdEnergy[ist] = g_sdEnergy[ist+1];
				g_sdEnergy[ist+1] = w;
			}
		}
	}


	// --------------------  utility  --------------------

	function innerProduct(ist,jst) {
		const nnx=g_NNx, nny=g_NNy;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				s += g_sdState[ist][i][j]*g_sdState[jst][i][j];
			}
		}
		return s*g_dx*g_dy;
	}

	function normalizeState(ist) {
		const nnx=g_NNx, nny=g_NNy;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				s += g_sdState[ist][i][j]*g_sdState[ist][i][j];
			}
		}
		const a = Math.sqrt(1.0/(s*g_dx*g_dy));
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_sdState[ist][i][j] = a*g_sdState[ist][i][j];
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( stateMax, vIndex )
		evolve:			SDiteration,			// SDiteration( stateMax, iterMax )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx ]; },
		getNow:			function() { return [ g_iterCount, g_sdEnergy[0] ]; },
		getStEnergy:	function(ist) { return g_sdEnergy[ist]; },
		getState:		function(ist,i,j) { return g_sdState[ist][i][j]; },
		getVext:		function(i,j) { return g_vv[i][j]; },
	};

})(); // ====================  periodicPSD2D end  ====================


const js018 = (function(){ // ====================  js Module  ====================

	const theModule = periodicPSD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_stateMax = 10;
	let v_vIndex = 0; // 0:parabollic, 1:deep well, 2:shallow well, 3:rect. well, 4:ring
	let v_iterMax = 1;

	let p_NNx, p_NNy, p_dx; // = theModule.getSysParam();
	let iterCount, groundStateEnergy;
	let stateEnergyList = [];

	let dispMode = 3;
	let dispState = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_stateMax, v_vIndex );
			[ p_NNx, p_NNy, p_dx ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNy, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_stateMax, v_iterMax );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_stateMax, v_iterMax );
		}
		setStateEnergyList();

		draw( ctx, dispMode, dispState );

		requestAnimationFrame(animate);
	}

  function setStateEnergyList() {
    for (let i=0; i<v_stateMax; i++) {
      stateEnergyList[i] = theModule.getStEnergy(i);
    }
  }


	// --------------------  draw  --------------------

	function draw( ctx, dispMode, dispState ) {
		const stateFunc = function(i,j) { return 10.0*theModule.getState(dispState,i,j); };
		const densFunc = function(i,j) { const phi = theModule.getState(dispState,i,j); return 20.0*phi*phi; };
		const VextFunc = function(i,j) { return 0.4*theModule.getVext(i,j); };

		const xp = 20, yp = 60, xTextPos= 300, yTextPos= 20; // in pixel
		[ iterCount, groundStateEnergy ] = theModule.getNow();

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// state table
		if ( dispMode<=1 ) {
			for (let ist=v_stateMax-1; ist>=0; ist--) {
				ctx.fillStyle = "hsl("+ist*30+",100%,50%)";
				ctx.fillText("E["+ist+"]="+theModule.getStEnergy(ist).toFixed(6), 300, 300-ist*20);
			}
		}

		ctx.strokeStyle = "#888888";
		if (dispMode==0) {
			ctx.strokeRect( xp, yp, 256, 300 );
			drawAlongXaxis( ctx, v_stateMax, xp, 300 );
		} else if (dispMode==1) {
			ctx.strokeRect( xp, yp, 256, 256 );
			drawPh( ctx, dispState, xp, yp);

		}  else if (dispMode==2) {
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, VextFunc, "#00aa00", 1 );
			drawText(ctx, "Vext(x,y)", xp, yCanvasSize-30);

		} else if (dispMode==3) {
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, stateFunc, 0.5, 1 );
			drawText(ctx, "state = | "+dispState+" >", xp, yCanvasSize-30);
			drawText(ctx, "state energy = "+theModule.getStEnergy(dispState).toFixed(6), 260, yCanvasSize-30);

		} else if (dispMode==4) {
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, densFunc, 0.5, 1 );
			drawText(ctx, "| state(x,y) |^2 ", xp, yCanvasSize-30);
			drawText(ctx, "state energy = "+theModule.getStEnergy(dispState).toFixed(6), 260, yCanvasSize-30);

		} else if (dispMode==5) {
			const zFunc = function(i,j) { return stateFunc(i,j) + VextFunc(i,j) };
			const colorFunc = function(i,j) {
				const z = stateFunc(i,j);
				const th = (18120 - Math.floor(180.0*z/g3d.cz0))%360;
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? `hsl(${th},100%,50%)` : `hsl(${th},30%,20%)`;
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );
			drawText(ctx, "state = | "+dispState+" > + Vext(x,y)", xp, yCanvasSize-30);
			drawText(ctx, "state energy = "+theModule.getStEnergy(dispState).toFixed(6), 260, yCanvasSize-30);

		}  else if (dispMode==6) {
			const zFunc = function(i,j) { return densFunc(i,j) + VextFunc(i,j); };
			const colorFunc = function(i,j) {
				const z = densFunc(i,j);
				const th = (18120 - Math.floor(180.0*z/g3d.cz0))%360;
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? `hsl(${th},100%,50%)` : `hsl(${th},30%,20%)`;
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );
			drawText(ctx, "| state(x,y) |^2 + Vext(x,y)", xp, yCanvasSize-30);
			drawText(ctx, "state energy = "+theModule.getStEnergy(dispState).toFixed(6), 260, yCanvasSize-30);

		}

		ctx.fillStyle = "#888888";
		ctx.fillText(`periodic box = ${(p_NNx*p_dx).toFixed(1)} x ${(p_NNy*p_dx).toFixed(1)} (au)`, 20, yCanvasSize-10);
		ctx.fillText(`iter = ${iterCount}`, xTextPos, yCanvasSize-10);

		function drawText(ctx, str, xp, yp ) { ctx.fillStyle = "#888888"; ctx.fillText( str, xp, yp ); };
	}

	function drawPh( ctx, ist, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const g = Math.floor(theModule.getVext(i,j)*10);
				const p = theModule.getState(ist,i,j)*1000;
				let r, b;
				if (p>=0) {
					r = Math.floor(p); if (r>255) r = 255;
					b = 0;
				} else {
					r=0;
					b = Math.floor(-p); if (b>255) b = 255;
				}
				ctx.fillStyle = "rgb("+r+","+g+","+b+")";
				ctx.fillRect(i*4+xp,(nny-j-1)*4+yp,4,4);
			}
		}
		ctx.fillStyle = "rgb(120,120,120)";
		ctx.fillText("state red:ph(x,y)>0 blue:ph(x,y)<0", xp, 30);
		ctx.fillText("potential green:Vext(x,y)", xp, 50);
	}

	function drawAlongXaxis( ctx, stateMax, xp, yp ) {
		drawLine( ctx, xp, yp, xp+(p_NNx-1)*4, yp, "rgb(120,120,120)" ); // base line
		drawV( ctx, xp, yp, 10, "rgb(0,160,0)" ); // Vext(x,0)
		for (let ist=stateMax-1; ist>=0; ist--) { // state(x,0)
			drawState( ctx, ist, xp, yp, 100, 20, "hsl("+ist*30+",100%,50%)" );
		}
		ctx.fillStyle = "rgb(120,120,120)";
		ctx.fillText("along x-axis", xp, 30);
		ctx.fillStyle = "rgb(0,160,0)";
		ctx.fillText("V(x,0)", xp+5, 90);
	}

	function drawState( ctx, ist, xp, yp, pmag, emag, color ) {
		const nnx=p_NNx,nny=p_NNy;
		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let i=1; i<nnx; i++) {
			ctx.lineTo(i*4+xp,yp-theModule.getState(ist,i,nny/2)*pmag-theModule.getStEnergy(ist)*emag);
		}
		ctx.stroke();
	}

	function drawV( ctx, xp, yp, vmag, color ) {
		const nnx=p_NNx, nny=p_NNy;
		ctx.strokeStyle = color; //"rgb(0,160,0)";
		ctx.beginPath();
		for (let i=1; i<nnx; i++) {
			ctx.lineTo(i*4+xp,yp-theModule.getVext(i,nny/2)*vmag);
		}
		ctx.stroke();
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;


	// --------------------  dom control

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_vIndex = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setDispState() {
		dispState = 0 + document.getElementById("slct_dispState").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-15,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_vIndex = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispState( ist ) {
    dispState = ist;
    document.getElementById("slct_dispState").selectedIndex = ist;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ iterCount, groundStateEnergy ];
  }

  function pygetStateEnergyList() {
    return stateEnergyList;
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setDispMode:	setDispMode,	// setDispMode()
		setDispState:	setDispState,	// setDispState()
		viewHome:		viewHome,		// viewHome()

	  breakLoop: breakLoop, // breakLoop();
	  pysetTheme: pysetTheme, // pysetTheme( theme )
		pysetDispState: pysetDispState, // pysetDispState( ist )
	  pysetDispMode: pysetDispMode, // pysetDispMode( mode )
	  pygetData: pygetData, // pygetData( pyMsg ) : return [ iterCount, groundStateEnergy ]
	  pygetStateEnergyList: pygetStateEnergyList, // pygetStateEnergyList()
	};

})(); // ====================  js018 module end  ====================


const js = js018;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js018] periodic steepest descent method 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>potential:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>in periodic box</option>
<option>in checker potential</option>
<option>quasi-1D periodic</option>
</select>
    <span style="margin-right: 40px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>along x-axis</option><option>state(x,y)</option>
<option>grid2D: Vext(x,y)</option><option selected>grid2D: state(x,y)</option>
<option>grid2D: | state(x,y) |^2</option><option>grid2D: (state+Vext)(x,y)</option>
<option>grid2D: ( | state(x,y) |^2 + Vext)(x,y)</option>
</select>
	<span style="margin-right: 20px;"></span>
<label>state =</label>
<select id="slct_dispState" onChange="js.setDispState()">
<option>0</option><option>1</option><option>2</option><option>3</option>
<option>4</option><option>5</option><option>6</option><option>7</option>
<option>8</option><option>9</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def



In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ iterCount, groundStateEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  print(
    "i = {:>2},  ".format(i),
    "iter count = {:>4}, ".format(iterCount),
    "ground state energy = {:>9.6f} (au) ".format(groundStateEnergy) )
  time.sleep(2)
#

# print state energy list
stateEnergyList = eval_js( 'js.pygetStateEnergyList()' )
g6List = [ float( '{:.6g}'.format(num) ) for num in stateEnergyList ]
print("")
print( "state energy list :", g6List )

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change potential, dispState

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# change theme and dispState
potentialList = [ 'in periodic box', 'in checker potential', 'quasi-1D periodic' ]
dispStateList = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
for theme in [ 0, 1, 2 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", potentialList[theme], "--" )
  for ist in dispStateList:
    eval_js( 'js.pysetDispState({})'.format(ist) )
    print( "-- -- disp state:", ist, "--" )
    time.sleep(3)
    # get data and print
    [ iterCount, groundStateEnergy ] = eval_js( 'js.pygetData(0)' )
    print(
      "\t iter count = {:>4}".format(iterCount),
      ", ground state energy = {:>10.6f} (au)".format(groundStateEnergy) )
    time.sleep(2)
  # print state energy list
  stateEnergyList = eval_js( 'js.pygetStateEnergyList()' )
  g6List = [ float( '{:.6g}'.format(num) ) for num in stateEnergyList ]
  print( "-- state energy list :", g6List )
  print("")

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")